In [1]:
import pandas as pd
import requests
import time
from pathlib import Path

In [2]:
MODEL = "model-1"
EVAL_FILE = "../../data/processed/bdm-corpus-2/stage-5/test_filtered.csv"
RESULT_FILE = "../../results/exp002/results.csv"
TEMPERATURE = 0.0

In [3]:
df = pd.read_csv(EVAL_FILE)
len(df)

200

In [4]:
def build_prompt(headline: str) -> str:
    return (
        f"{headline.strip()}\n\n"
        f"Projetar o impacto no valor do BRL com exatamente um rótulo: 'Em alta' ou 'Em baixa'."
    )

In [5]:
# === Step 3: Query Ollama ===
def query_ollama(model, prompt):
    res = requests.post("http://localhost:11434/api/chat", json={
        "model": model,
        "temperature": TEMPERATURE,
        "stream": False,
        "messages": [{"role": "user", "content": prompt}]
    })
    res.raise_for_status()
    return res.json()["message"]["content"].strip()

In [7]:
# === Step 4: Run prediction loop ===
predictions = []
for i, row in df.iterrows():
    prompt = build_prompt(row["Headline"])
    try:
        out = query_ollama(MODEL, prompt)
    except Exception as e:
        out = "Erro"
    predictions.append(out)
    time.sleep(0.25)  # reduce overload risk

In [18]:
import re
predictions = [
    re.sub(r'\s*\n+.*$', '', p.strip().lstrip("'\"").rstrip("'\"").strip())
    for p in predictions
]

In [19]:
predictions

['Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em alta',
 'Em baixa',
 'Em alta',
 'E

In [20]:
df["Prediction"] = predictions
df.to_csv(RESULT_FILE, index=False)
print(f"Saved predictions to {RESULT_FILE}")

Saved predictions to ../../results/exp002/results.csv


### Predictions

In [ ]:
from sklearn.metrics import classification_report

# Reload the results file
df = pd.read_csv("../../results/exp002/results_filtered.csv")

df['Prediction'] = df['Prediction'].map({
    'Em baixa': 'Increase',
    'Em alta': 'Decrease'
})

# Report
report = classification_report(
    df["Direction"], df["Prediction"],
    labels=["Increase", "Decrease"],
    target_names=["Increase", "Decrease"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

    Increase      0.645     0.331     0.437       121
    Decrease      0.413     0.722     0.525        79

    accuracy                          0.485       200
   macro avg      0.529     0.526     0.481       200
weighted avg      0.553     0.485     0.472       200



In [ ]:
from sklearn.metrics import confusion_matrix
'''
[[TP FN]
 [FP TN]]
'''
cm = confusion_matrix(df["Direction"], df["Prediction"], labels=["Increase", "Decrease"])
print(cm)

[[ 79 150]
 [ 46 125]]
